In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, col
from pyspark.sql.types import *

import re
import time

In [2]:
IN_PATH = '/data/data/raw_data/'
OUT_PATH = '/data/data/parsed_data/'

# IN_FILE = 'wiki_dump_500.xml'
# IN_FILE = 'fiction_category.xml'
IN_FILE = 'complete_dump.xml'
OUT_FILE = 'output.csv'

In [3]:
spark = SparkSession.builder.appName("test").getOrCreate()

# spark = SparkSession.builder.master("spark://sparkmaster:7077").appName("parsing").getOrCreate()
# sc = spark.sparkContext

/opt/bitnami/spark/python/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
root = 'mediawiki'
row = 'page'

schema = StructType([StructField('id', StringType(), True),
                        StructField('title', StringType(), True),
                        StructField('revision', StructType([StructField('text', StringType(), True)]))])

df = spark.read.format('com.databricks.spark.xml')\
    .options(rootTag=root)\
    .options(rowTag=row)\
    .schema(schema)\
    .load(IN_PATH + IN_FILE)

In [5]:
df = df.withColumn("revision", col("revision").cast("String"))
df = df.withColumnRenamed("revision", "text")

In [6]:
def clean_data(data):

    data = data.lower()
    data = re.sub(r'[\"\[\]]', ' ', data)
    data = re.sub(r' +', ' ', data)
    data = re.sub(r'[^a-z0-9|()*=]+', ' ', data)

    return data

In [7]:
def extract_infobox_data(page):
    infobox_data = []
    result = 'production info | '
    
    movie_data = [
    'director',
    'producer',
    'screenplay',
    'writer',
    'story', 
    'music', 
    'cinematography', 
    'editing'
    ]

#     infobox_data.append(re.findall('(?i)name *=.*?(?=\|)\|', page))
    
    for item in movie_data:
        infobox_data.append(re.findall('(?i)'+item+' *=.*?(?=\])\]', page) + "|")

    for item in infobox_data:
        result += ''.join(item)
        
    return result

In [8]:
def get_name(line):
    name = re.match(r"^.*\[(.*)\]\].*$", line)
    return name.group(1)


def init_movie_info(movie_info, wanted_data):    
    for x in wanted_data:
        movie_info[x] = []
    return movie_info
    
    
def get_movie_info(page):
    movie_info = {}
    
    wanted_data = [
        'director',
        'producer',
        'screenplay',
        'writer',
        'story', 
        'music', 
        'cinematography', 
        'editing'
        ]
    
    for x in wanted_data:
        movie_info[x] = []
    
    index = 0
    
    text_lines = page.splitlines()

    for line in text_lines:
        line = line.strip()
            
        if "{{infobox film" in line.lower():
            index += 1
            try:
                while text_lines[index] != " ":

                    if text_lines[index]:
                        if text_lines[index][0] == "'":
                            break

                    for item in wanted_data:
                        line = text_lines[index].replace(" ", "").lower()

                        if "|" + item in line and "plainlist" not in line:
                            if "[[" not in text_lines[index]:
                                try:
                                    name = re.match(r"^.*\= (.*).*$", text_lines[index])
                                    movie_info[item].append(name.group(1))
                                except:
                                    continue    
                            else:
                                movie_info[item].append(get_name(text_lines[index]))
                    
                        if "|" + item in line and "plainlist" in line:

                            i = index + 1 
                            while "}}" not in text_lines[i]:  
                                if "[[" not in text_lines[i]:
                                    movie_info[item].append(text_lines[i][3:])         
                                else:
                                    movie_info[item].append(get_name(text_lines[i]))

                                i += 1
                    index += 1
        
                tmp = []
                for item in movie_info:
                    tmp.append(item + "=")
                    for name in movie_info[item]:
                        if len(movie_info[item]) > 1:
                            tmp.append(name + ", ")
                        else:
                            tmp.append(name)
                    tmp.append("|")
                
                result = ""
                for item in tmp:
                    result += ''.join(item)
            
            except:
                result = ""
                
            return result

        index += 1
    return

In [9]:
def get_name(line):
        name = re.match(r"^.*\[(.*)\]\].*$", line)
        return name.group(1)
    
def get_infobox_data(page):
    index = 0
    text_lines = page.splitlines()
    
    buffer = []

    for line in text_lines:
        line = line.strip()
            
        if re.search('(?i){{Infobox *film', line):
            while text_lines[index] and  text_lines[index] != " ":

                if text_lines[index]:
                    if text_lines[index][0] == "'":
                        break
                        
                buffer.append(text_lines[index])
                
                if (index+1) >= len(text_lines):
                    break
                index += 1
        index += 1
        
    result = ' '.join(buffer).replace('\n', ' ')

#     infobox_data = extract_infobox_data(result)
    infobox_data = get_movie_info(text_lines)
    return infobox_data

In [10]:
def get_movie_cast(page):
    cast = []
    result = '|movie cast|'
    
    page = re.sub('[^A-Za-z0-9]+', ' ', page)
    cast.append(re.findall('[A-Z][A-Za-z]* [A-Z][A-Za-z]* as [A-Z][A-Za-z]* [A-Z][A-Za-z]*', page))

    for item in cast:
        result += ''.join(item)
        
    return result

In [11]:
def is_movie_page(page):
    if re.search('(?i){{Infobox *film', str(page)):
        return True
    return False


def parse_wiki_page(page):
    if is_movie_page(page):
        movie_info = []
        
#         infobox_data = get_infobox_data(page)
#         infobox_data = clean_data(infobox_data)

        infobox_data = get_movie_info(page)
        cast = get_movie_cast(page)
        
        movie_info.append(infobox_data)
        movie_info.append(cast)
        
        return movie_info

    return None

In [12]:
def format_time(seconds):
    min, sec = divmod(seconds, 60)
    hour, min = divmod(min, 60)
    
    return "%d:%02d:%02d" % (hour, min, sec)

In [14]:
start = time.time()

my_udf = UserDefinedFunction(parse_wiki_page, StringType())

df_new = df.withColumn('text', my_udf('text'))
df_new = df_new.na.drop()

try:
    df_new.repartition(10).write.format('com.databricks.spark.csv').mode("overwrite") \
    .save(OUT_PATH + "test", header = 'true')
        
except Exception as e:
    print(e)
    exit(1)
    
end = time.time()
print('Elapsed time: ', format_time(end - start))

Elapsed time:  2:00:24
